# Yelp Data Challenge - Data Preprocessing

BitTiger DS501

Jun 2017

## Dataset Introduction

[Yelp Dataset Challenge](https://www.yelp.com/dataset_challenge)

The Challenge Dataset:

    4.1M reviews and 947K tips by 1M users for 144K businesses
    1.1M business attributes, e.g., hours, parking availability, ambience.
    Aggregated check-ins over time for each of the 125K businesses
    200,000 pictures from the included businesses

Cities:

    U.K.: Edinburgh
    Germany: Karlsruhe
    Canada: Montreal and Waterloo
    U.S.: Pittsburgh, Charlotte, Urbana-Champaign, Phoenix, Las Vegas, Madison, Cleveland

Files:

    yelp_academic_dataset_business.json
    yelp_academic_dataset_checkin.json
    yelp_academic_dataset_review.json
    yelp_academic_dataset_tip.json
    yelp_academic_dataset_user.json

Notes on the Dataset

    Each file is composed of a single object type, one json-object per-line.
    Take a look at some examples to get you started: https://github.com/Yelp/dataset-examples.



## Read data from file and load to Pandas DataFrame

**Warning**: Loading all the 1.8 GB data into Pandas at a time takes long time and a lot of memory!

In [42]:
import json
from collections import defaultdict

In [43]:
import pandas as pd

In [47]:
file_business, file_checkin, file_review, file_tip, file_user = [
    'yelp_academic_dataset_business.json',
    'yelp_academic_dataset_checkin.json',
    'yelp_academic_dataset_review.json',
    'yelp_academic_dataset_tip.json',
    'yelp_academic_dataset_user.json'
]

#### Business Data

In [50]:
with open(file_business) as f:
    df_business = pd.DataFrame(json.loads(line) for line in f)

In [51]:
df_business.head(2)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB
1,,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,,89002,3,4.5,NV


In [8]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 16 columns):
address         100 non-null object
attributes      87 non-null object
business_id     100 non-null object
categories      100 non-null object
city            100 non-null object
hours           68 non-null object
is_open         100 non-null int64
latitude        100 non-null float64
longitude       100 non-null float64
name            100 non-null object
neighborhood    100 non-null object
postal_code     100 non-null object
review_count    100 non-null int64
stars           100 non-null float64
state           100 non-null object
type            100 non-null object
dtypes: float64(3), int64(2), object(11)
memory usage: 12.6+ KB


#### Checkin Data

In [52]:
with open(file_checkin) as f:
    df_checkin = pd.DataFrame(json.loads(line) for line in f)
df_checkin.head(2)

,business_id,time
0,7KPBkxAOEtb3QeIL9PEErg,"{'Fri-0': 2, 'Sat-0': 1, 'Sun-0': 1, 'Wed-0': ..."
1,kREVIrSBbtqBhIYkTccQUg,"{'Mon-13': 1, 'Thu-13': 1, 'Sat-16': 1, 'Wed-1..."


#### Review Data

In [ ]:
with open(file_review) as f:
    df_review = pd.DataFrame(json.loads(line) for line in f)
df_review.head(2)

#### Tip Data

In [ ]:
with open(file_tip) as f:
     df_tip = pd.DataFrame(json.loads(line) for line in f)
df_tip.head(2)

#### User Data

In [ ]:
 with open(file_user) as f:
    df_user = pd.DataFrame(json.loads(line) for line in f)
df_user.head(2)

## Filter data by city and category

#### Create filters/masks

* create filters that selects business 
    * that are located in "Las Vegas"
    * that contains "Restaurants" in their category (You may need to filter null categories first)

In [ ]:
# Create Pandas DataFrame filters
df_business = pd.read_json('yelp_academic_dataset_business.json', lines=True)
df_business = df_business[df_business['city'] == 'Las Vegas']

In [ ]:
# Create filtered DataFrame, and name it df_filtered
# retain only "Restaurants"
df_filtered = df_business[df_business['categories'].apply(lambda x: 'Restaurants' in x)].reset_index(drop=True)[['business_id', 'stars', 'name','categories']]

# round stars and keep the results in column "stars_round"
df_filtered['stars_round'] = df_business_restaurants['stars'].apply(round)

df_filtered.head()

#### Keep relevant columns

* only keep some useful columns
    * business_id
    * name
    * categories
    * stars

In [15]:
selected_features = [u'business_id', u'name', u'categories', u'stars']

In [16]:
# Make a DataFrame that contains only the abovementioned columns, and name it as df_selected_business
df_selected_business = df_business[df_business['categories'].apply(lambda x: 'Restaurants' in x)].reset_index(drop=True)[['business_id', 'stars', 'name','categories']]

In [17]:
# Rename the column name "stars" to "avg_stars" to avoid naming conflicts with review dataset
df_selected_business.rename(columns = {"stars":"avg_stars"})

In [18]:
# Inspect your DataFrame
df_selected_business.head()

#### Save results to csv files

In [19]:
# Save to ./data/selected_business.csv for your next task
df_selected_business.to_csv('selected_business.csv',header = True)

In [20]:
# Try reload the csv file to check if everything works fine
df_selected_business_csv = pd.read_csv('selected_business.csv')

### Use the "business_id" column to filter review data

* We want to make a DataFrame that contain and only contain the reviews about the business entities we just obtained

#### Load review dataset

In [21]:
with open(file_review) as f:
    df_review = pd.DataFrame(json.loads(line) for line in f)
df_review.head(2)

,business_id,cool,date,funny,review_id,stars,text,type,useful,user_id
0,2aFiy99vNLklCx3T_tGS9A,0,2011-10-10,0,NxL8SIC5yqOdnlXCg18IBg,5,If you enjoy service by someone who is as comp...,review,0,KpkOkG6RIf4Ra25Lhhxf1A
1,2aFiy99vNLklCx3T_tGS9A,0,2010-12-29,0,pXbbIgOXvLuTi_SPs1hQEQ,5,After being on the phone with Verizon Wireless...,review,1,bQ7fQq1otn9hKX-gXRsrgA


#### Prepare dataframes to be joined, - on business_id

In [22]:
# Prepare the business dataframe and set index to column "business_id", and name it as df_left
df_business_2 = pd.read_json('yelp_academic_dataset_business.json', lines=True, index = 'business_id')
pd.Index.rename("df_left", inplace = False)

In [23]:
# Prepare the review dataframe and set index to column "business_id", and name it as df_right
df_business_3 = pd.read_json('yelp_academic_dataset_review.json', lines=True, index = 'business_id')
pd.Index.rename("df_right",inplace = False)

#### Join! and reset index

In [24]:
# Join df_left and df_right. What type of join?
#Join using index with left join
df_merge = pd.merge(df_business_2, df_business_3, how = 'left')

In [25]:
# You may want to reset the index 
df_merge.reset_index()

#### We further filter data by date, e.g. keep comments from last 2 years

* Otherwise your laptop may crush on memory when running machine learning algorithms
* Purposefully ignoring the reviews made too long time ago

In [26]:
# Make a filter that selects date after 2015-01-20
df_final = pd.read_json('yelp_academic_dataset_review.json', lines=True)
df_final = df_business_3[df_business_3['date'] >= '2015-01-20']

In [27]:
# Filter the joined DataFrame and name it as df_final
df_final.head()

#### Take a glance at the final dataset

* Do more EDA here as you like!

In [28]:
import matplotlib.pyplot as plt

% matplotlib inline

In [29]:
# e.g. calculate counts of reviews per business entity, and plot it

# group businesses by stars
group_by_stars = df_final.groupby('stars_round')

# frequency of cities corresponding to stars
city_freq_of_stars = {}

num_cities = len(df_business_restaurants['city'].unique())

# increase the value of maximum rows in order to show all frequency results later.
# +1 is for the table header.
pd.set_option('display.max_rows', num_cities + 1)

for star in group_by_stars.groups:
    business_of_star_df = group_by_stars.get_group(star)
    
    city_freq = {}
    num_business = len(business_of_star_df)
    
    # Now group them by city
    city_of_business = business_of_star_df.groupby('city')
    for city in city_of_business.groups:
        # we use the add-one or Laplace smoothing
        city_freq[city] = (len(city_of_business.get_group(city)) + 1.0) / (num_business + num_cities)
    
    # this value is for cities not in the a specify "group of star",
    # e.g. city "glendale" is not in group of star 1
    city_freq['default'] = 1.0 / (num_business + num_cities)
    
    city_freq_of_stars[star] = city_freq
    
city_freq_of_stars_df = pd.DataFrame.from_dict(city_freq_of_stars)

for c in city_freq_of_stars_df.columns.values:
    city_freq_of_stars_df[c] = city_freq_of_stars_df[c].fillna(city_freq_of_stars_df[c]['default'])

city_freq_of_stars_df

## Save your preprocessed dataset to csv file

* Respect your laptop's hard work! You don't want to make it run everything again.

In [30]:
# Save to ./data/last_2_years_restaurant_reviews.csv for your next task
df_final.to_csv("../data/last_2_years_restaurant_reviews.csv")